In [53]:
from typing import Annotated
from pydantic import BaseModel, ConfigDict, Field, AfterValidator

In [2]:
class Point2D(BaseModel):
    x: float = 0
    y: float = 0

In [3]:
class Circle2D(BaseModel):
    center: Point2D
    radius: float = Field(default=1, gt=0)

In [4]:
c = Circle2D(center=Point2D(x=1, y=1), radius=2)

In [5]:
c

Circle2D(center=Point2D(x=1.0, y=1.0), radius=2.0)

In [6]:
c = Circle2D(center=(1, 1), radius=2)

ValidationError: 1 validation error for Circle2D
center
  Input should be a valid dictionary or instance of Point2D [type=model_type, input_value=(1, 1), input_type=tuple]
    For further information visit https://errors.pydantic.dev/2.8/v/model_type

In [7]:
c = Circle2D(center={'x': 1, 'y': 1}, radius=2)

In [8]:
c.model_dump()

{'center': {'x': 1.0, 'y': 1.0}, 'radius': 2.0}

In [11]:
print(c.model_dump_json(indent=2))

{
  "center": {
    "x": 1.0,
    "y": 1.0
  },
  "radius": 2.0
}


In [12]:
data = {'center': {'x': 5, 'y': -5}, 'radius': 10}

In [13]:
c = Circle2D.model_validate(data)

In [18]:
data = '{"center": {"x": 5, "y": -5}, "radius": 10}'

In [19]:
Circle2D.model_validate_json(data)

Circle2D(center=Point2D(x=5.0, y=-5.0), radius=10.0)

In [21]:
c.center

Point2D(x=5.0, y=-5.0)

In [22]:
c.center.model_dump()

{'x': 5.0, 'y': -5.0}

In [23]:
json_data = """
{
    "firstName": "David",
    "lastName": "Hilbert",
    "contactInfo": {
        "email": "d.hilbert@spectral-theory.com",
        "homePhone": {
            "countryCode": 49,
            "areaCode": 551,
            "localPhoneNumber": 123456789
        }
    },
    "personalInfo": {
        "nationality": "German",
        "born": {
            "date": "1862-01-23",
            "place": {
                "city": "Konigsberg",
                "country": "Prussia"
            }
        },
        "died": {
            "date": "1943-02-14",
            "place": {
                "city": "Gottingen",
                "country": "Germany"
            }
        }
    },
    "awards": ["Lobachevsky Prize", "Bolyai Prize", "ForMemRS"],
    "notableStudents": ["von Neumann", "Weyl", "Courant", "Zermelo"]
}
"""

In [25]:
from pydantic import EmailStr, PastDate
from pydantic.alias_generators import to_camel

In [26]:
class ContactInfo(BaseModel):
    model_config = ConfigDict(
        extra="ignore"
    )
    email: EmailStr | None = None

In [27]:
class PlaceInfo(BaseModel):
    city: str
    country: str

In [32]:
class PlaceDateInfo(BaseModel):
    model_config = ConfigDict(
        populate_by_name=True
    )
    
    date_: PastDate = Field(alias="date")
    place: PlaceInfo

In [33]:
class PersonalInfo(BaseModel):
    model_config = ConfigDict(
        extra="ignore"
    )
    nationality: str
    born: PlaceDateInfo

In [34]:
class Person(BaseModel):
    model_config = ConfigDict(
        alias_generator=to_camel,
        populate_by_name=True,
        extra="ignore"
    )
    first_name: str
    last_name: str
    contact_info: ContactInfo
    personal_info: PersonalInfo
    notable_students: list[str] = []

In [35]:
p = Person.model_validate_json(json_data)

In [36]:
p

Person(first_name='David', last_name='Hilbert', contact_info=ContactInfo(email='d.hilbert@spectral-theory.com'), personal_info=PersonalInfo(nationality='German', born=PlaceDateInfo(date_=datetime.date(1862, 1, 23), place=PlaceInfo(city='Konigsberg', country='Prussia'))), notable_students=['von Neumann', 'Weyl', 'Courant', 'Zermelo'])

In [37]:
p.model_dump()

{'first_name': 'David',
 'last_name': 'Hilbert',
 'contact_info': {'email': 'd.hilbert@spectral-theory.com'},
 'personal_info': {'nationality': 'German',
  'born': {'date_': datetime.date(1862, 1, 23),
   'place': {'city': 'Konigsberg', 'country': 'Prussia'}}},
 'notable_students': ['von Neumann', 'Weyl', 'Courant', 'Zermelo']}

In [38]:
from pprint import pprint

In [41]:
pprint(p.model_dump())

{'contact_info': {'email': 'd.hilbert@spectral-theory.com'},
 'first_name': 'David',
 'last_name': 'Hilbert',
 'notable_students': ['von Neumann', 'Weyl', 'Courant', 'Zermelo'],
 'personal_info': {'born': {'date_': datetime.date(1862, 1, 23),
                            'place': {'city': 'Konigsberg',
                                      'country': 'Prussia'}},
                   'nationality': 'German'}}


In [45]:
print(p.model_dump_json(indent=2))

{
  "first_name": "David",
  "last_name": "Hilbert",
  "contact_info": {
    "email": "d.hilbert@spectral-theory.com"
  },
  "personal_info": {
    "nationality": "German",
    "born": {
      "date_": "1862-01-23",
      "place": {
        "city": "Konigsberg",
        "country": "Prussia"
      }
    }
  },
  "notable_students": [
    "von Neumann",
    "Weyl",
    "Courant",
    "Zermelo"
  ]
}


In [72]:
SortedStrList = Annotated[
    list[str],
    AfterValidator(lambda x: sorted(x, key=str.casefold))
]

In [73]:
class Person(BaseModel):
    model_config = ConfigDict(
        alias_generator=to_camel,
        populate_by_name=True,
        extra="ignore"
    )
    first_name: str
    last_name: str
    contact_info: ContactInfo
    personal_info: PersonalInfo
    notable_students: SortedStrList = []

In [74]:
p = Person.model_validate_json(json_data)

In [75]:
p.model_dump()

{'first_name': 'David',
 'last_name': 'Hilbert',
 'contact_info': {'email': 'd.hilbert@spectral-theory.com'},
 'personal_info': {'nationality': 'German',
  'born': {'date_': datetime.date(1862, 1, 23),
   'place': {'city': 'Konigsberg', 'country': 'Prussia'}}},
 'notable_students': ['Courant', 'von Neumann', 'Weyl', 'Zermelo']}